In [ ]:
import Pkg; Pkg.add(url="https://github.com/JuliaStats/StatsFuns.jl", rev="an/nopdf")
Pkg.add("SpecialFunctions")
using StatsFuns: betapdf,betalogpdf,betacdf, betaccdf,betalogcdf,betalogccdf,betainvcdf,betainvccdf,xlogy, log1p,logbeta
using SpecialFunctions: beta_inc

In [ ]:
function betalogpdf2(α::T, β::T, x::T) where T<:Real
    return if (x < 1 || x > 1)
        -Inf
    elseif x <= 0.5
        xlogy(α - 1, x) + (β - 1) * log1p(-x) - logbeta(α, β)
    else
        betalogpdf2(β, α, 1 - x)
    end
end

function betalogcdf2(α::Float64, β::Float64, x::Float64)
    if x < 0
        return -Inf
    elseif x > 1
        return 0
    else
        p, q = beta_inc(α, β, x)
        if p < eps(one(p))
            # see https://dlmf.nist.gov/8.17#E7
            return -log(α) + α*log(x) + log(_₂F₁(promote(α, 1 - β, α + 1, x)...)) - logbeta(α, β)
        elseif p <= 0.7
            return log(p)
        else
            return log1p(-q)
        end
    end
end
betalogcdf2(α::Real, β::Real, x::Real) = betalogcdf2(promote(float(α), β, x)...)
beta_inc2(α::Real, β::Real, x::Real) = beta_inc(promote(float(α), β, x)...)

In [ ]:
α=1.1;β=2.3;x=2.0
betalogpdf2(α,β,x)
betalogcdf2(α,β,x)


In [ ]:
using ForwardDiff, Zygote

In [ ]:
ForwardDiff.derivative(x->beta_inc2(α,β,x),0.3)

In [ ]:
import Pkg; Pkg.add.(["ModelingToolkit","GalacticOptim","Optim","ForwardDiff","Zygote","NonlinearSolve","RuntimeGeneratedFunctions"])

In [ ]:
using ModelingToolkit, GalacticOptim, Optim, ForwardDiff, Zygote, NonlinearSolve,RuntimeGeneratedFunctions,LinearAlgebra

In [ ]:
import ModelingToolkit: calculate_gradient, generate_gradient
function calculate_gradient(sys::NonlinearSystem)
    expand_derivatives.(gradient(equations(sys), states(sys)))
end

function generate_gradient(sys::NonlinearSystem, vs = states(sys), ps = parameters(sys); kwargs...)
    grad = calculate_gradient(sys)
    return build_function(grad, vs, ps;
                          conv = AbstractSysToExpr(sys),kwargs...)
end

In [ ]:
using SciMLBase,DiffEqBase,Symbolics, ModelingToolkit, GalacticOptim, Optim, ForwardDiff, Zygote, NonlinearSolve,RuntimeGeneratedFunctions,LinearAlgebra 
using UnPack: @unpack

module NonLinProbPrecompile
    using ModelingToolkit, LinearAlgebra

    function system(; kwargs...)
        # Define some variables
        A = Matrix(I,2,2); b = rand(2)
        @variables z[1:5]
        @parameters p[1:2]
        zcat = vcat(z...);
        pcat = vcat(p...);
    
        # Define a system of nonlinear equations
        ceq = vcat(0 .~ A*zcat[1:2].^4 - pcat, 0 .~ A*zcat[3:4] - pcat + vcat(z[5],z[2]))
        ns = NonlinearSystem(ceq,z,p)
        return generate_function(ns,z,p)
    end
    # Setting eval_expression=false and eval_module=[this module] will ensure
    # the RGFs are put into our own cache, initialised below.
    using RuntimeGeneratedFunctions
    RuntimeGeneratedFunctions.init(@__MODULE__)
    const f_noeval_good = system(; eval_expression=false, eval_module=@__MODULE__)
end

In [ ]:
f = eval(NonLinProbPrecompile.f_noeval_good[1])
z0=ones(5)
p0=[2.2,2.3]
@variables z[1:5]
@parameters p[1:2]
f_expr = f(z,p)
eq = 0 .~ f(z,p)
ns = NonlinearSystem(eq,z,p; name=:random_name, defaults=merge(Dict(vcat(z) .=> z0),Dict(vcat(p) .=> p0)) ) #NonlinearSystem(eq,z,p)
prob_ns = NonlinearProblem(ns,z0,p0; check_length=false,jac = true, sparse=true,checkbounds = false, linenumbers = false) #NonlinearProblem(ns,z0,p0; check_length=false)
# NonlinearFunction(ns,z0,p0)
@show states(ns);
@show parameters(ns);

In [ ]:
sys_num = generate_function(ns) 
sys_sym = generate_function(ns,z,p) 
jac_num = generate_jacobian(ns)
jac_sym = generate_jacobian(ns,z,p) 
jac_sym_sp = Symbolics.sparsejacobian(f(z,p),states(ns))
hess_sym = ModelingToolkit.hessian(f(z,p)[1],states(ns); simplify=true) # hessian of 1st equation in f
hess_num = build_function(hess_sym, states(ns); expression = false, target = Symbolics.JuliaTarget())

sn = @eval eval(sys_num[1])
sn_iip = @eval eval(sys_num[2])
ss = @eval eval(sys_sym[1])

jn = @eval eval(jac_num[1])
jn_iip = @eval eval(jac_num[2])
js = @eval eval(jac_sym[1])
js_sp = jac_sym_sp

hn = @eval eval(hess_num[1])
hn_iip = @eval eval(hess_num[2])
hs = @eval eval(hess_sym)

In [ ]:
@show sn(z0,p0)
@show ss(z,p)
@show jn(z0,p0)
@show js(z,p)
@show js_sp
@show hn(z0,p0)
@show hs

sn_z0 = zeros(length(z0)) 
jn_z0 = zeros(length(eq),length(z0)) 
hn_z0 = zeros(length(z0),length(z0)) 
@show sn_iip(sn_z0,z0,p0)
@show jn_iip(jn_z0,z0,p0)
@show hn_iip(hn_z0,z0,p0)

@show sn_z0
@show jn_z0
@show hn_z0

In [ ]:
ns_tear = tearing(ns;simplify=true)
ns_tear = tearing(ns;simplify=false)
states(ns_tear)
parameters(ns_tear)
s = structure(ns_tear)
@unpack fullvars, vartype, varassoc, inv_varassoc, varmask, algeqs, graph, solvable_graph, assign, inv_assign, scc, partitions = s
@show jac_spcy = ModelingToolkit.jacobian_sparsity(ns);
@show jac_spcy.colptr;
@show jac_spcy.rowval;

In [ ]:
zcat = vcat(z...)
loss =  sum(zcat).^3
op_sys = OptimizationSystem(loss,z,p,name=:sys1)

In [ ]:
@show equations(op_sys);
@show states(op_sys);
@show parameters(op_sys);

In [ ]:
op_grad_sym = calculate_gradient(op_sys)
op_hess_sym = calculate_hessian(op_sys)

op_loss_num = generate_function(op_sys)
op_grad_num = generate_gradient(op_sys)
op_hess_num = generate_hessian(op_sys)
op_hess_spcy = ModelingToolkit.hessian_sparsity(op_sys)

In [ ]:
display(op_grad_sym)
display(op_hess_sym)

op_ln = @eval eval(op_loss_num)

op_gn = @eval eval(op_grad_num[1])
op_gn_iip = @eval eval(op_grad_num[2])

op_hess = @eval eval(op_hess_num[1])
op_hess_iip = @eval eval(op_hess_num[2])
op_hess_sp = op_hess_spcy


In [ ]:
@show op_ln(z0,p0)
@show op_gn(z0,p0)
gn0 = zeros(length(z0))
op_gn_iip(gn0,z0,p0)
@show gn0;
@show op_hess(z0,p0)
hess0 = zeros(length(z0),length(z0))
op_hess_iip(hess0,z0,p0)
@show hess0
@show display(op_hess_sp )

In [ ]:
prob = OptimizationProblem(op_sys,z0,p0,grad=true,hess=true, lb=zeros(size(z0)), ub=ones(size(z0)) )
sol = solve(prob,Optim.Newton())

In [ ]:
# equality_constraints not implemented yet?
op_eq = [0 ~ z[1]]
op_sys = OptimizationSystem(loss,z,p,name=:sys1,equality_constraints=op_eq)

In [ ]:
# lb and ub
opt_f = OptimizationFunction(op_ln,GalacticOptim.AutoZygote())
opt_prob = OptimizationProblem(opt_f, z0, p0 , lb=zeros(size(z0)), ub=ones(size(z0)) )
solve(opt_prob,Optim.BFGS())           

In [ ]:
# lb and ub
opt_f = OptimizationFunction(op_ln,GalacticOptim.AutoForwardDiff())
opt_prob = OptimizationProblem(opt_f, z0, p0 , lb=zeros(size(z0)), ub=ones(size(z0)) )
solve(opt_prob,IPNewton())

In [ ]:
                          grad=nothing,
                          hess=nothing,
                          hv=nothing,
                          cons=nothing,
                          cons_j=nothing,
                          cons_h=nothing)

In [ ]:
import Pkg; 
Pkg.resolve();Pkg.gc();Pkg.precompile()


In [ ]:
1

In [ ]:
prob = OptimizationProblem(op_sys,z0,p0,grad=true,hess=true, lb=zeros(size(z0)), ub=ones(size(z0)),lcons=0,ucons=0.1 )
sol = solve(prob,Optim.Newton())
#OptimizationProblemExpr(op_sys, z0,p0, lb=zeros(size(z0)), ub=ones(size(z0)) ,grad=true,hess=true)

In [ ]:
calculate_gradient(op_sys)
calculate_hessian(op_sys)
generate_gradient(op_sys)
generate_hessian(op_sys)
ModelingToolkit.hessian_sparsity(op_sys)

In [ ]:
using ModelingToolkit, SparseArrays, Test, GalacticOptim, Optim


equations(combinedsys)
states(combinedsys)
parameters(combinedsys)

calculate_gradient(combinedsys)
calculate_hessian(combinedsys)
generate_function(combinedsys)
generate_gradient(combinedsys)
generate_hessian(combinedsys)
ModelingToolkit.hessian_sparsity(combinedsys)

u0 = [
    sys1.x=>1.0
    sys1.y=>2.0
    sys2.x=>3.0
    sys2.y=>4.0
    z=>5.0
]
p = [
    sys1.a => 6.0
    sys1.b => 7.0
    sys2.a => 8.0
    sys2.b => 9.0
    β => 10.0
]

prob = OptimizationProblem(combinedsys,u0,p,grad=true)
sol = solve(prob,NelderMead())
@test sol.minimum < -1e5

prob2 = remake(prob,u0=sol.minimizer)
sol = solve(prob,BFGS(initial_stepnorm=0.0001),allow_f_increases=true)
@test sol.minimum < -1e8
sol = solve(prob2,BFGS(initial_stepnorm=0.0001),allow_f_increases=true)
@test sol.minimum < -1e9

rosenbrock(x, p) =  (p[1] - x[1])^2 + p[2] * (x[2] - x[1]^2)^2
x0 = zeros(2)
_p  = [1.0, 100.0]

f = OptimizationFunction(rosenbrock,ModelingToolkit.AutoModelingToolkit(),x0,_p,grad=true,hess=true)
prob = OptimizationProblem(f,x0,_p)
sol = solve(prob,Optim.Newton())